# Data preprocessing

In [1]:
import numpy as np
import pandas as pd
import re

In [ ]:
# full data is local uncommited storage, because of dataset size, you can find full data in cloud by link in README
df = pd.read_csv('/../full-data/full_cars_data.csv')

In [5]:
df.shape

(206656, 9)

In [6]:
df.head()

,price,title,year,mileage,engine,transmission,body_type,drive_type,color
0,NaN,Mercedes-Benz G-Класс 230 II (W463),1990,197000км,2.3 л / 125 л.с. / Бензин,автомат,внедорожник 3 дв.,полный,серый
1,NaN,Mercedes-Benz S-Класс 560 II (W126) Рестайлинг,1990,2800км,5.6 л / 279 л.с. / Бензин,автомат,седан,задний,коричневый
2,NaN,Toyota Carina V (T170),1990,300000км,1.6 л / 98 л.с. / Бензин,автомат,седан,передний,синий
3,189 700 ₽,Volkswagen Passat B3,1990,325000км,1.8 л / 90 л.с. / Бензин,механика,универсал 5 дв.,передний,красный
4,370 000 ₽,Nissan Terrano I,1990,224000км,2.7 л / 100 л.с. / Дизель,автомат,внедорожник 5 дв.,полный,зелёный


Целевая переменная здесь -- price.

Как мы видим, в колонке price есть пропуски. Давайте удалим все строки, в которых price = NaN

In [7]:
df = df.dropna(subset=['price'])

In [8]:
df['price'].isna().sum()

0

Теперь пропусков в price у нас нет.

Давайте посмотрим, какие единицы измерения есть в столбцах:

In [9]:
print("Различные валюты:", df['price'].apply(lambda x: re.sub(r'[\d\s]+', '', x)).unique())
print("Различные единицы измерения расстояния:", df['mileage'].apply(lambda x: re.sub(r'[\d\s]+', '', x)).unique())
print("Различные единицы измерения двигателя:", df['engine'].apply(lambda x: re.sub(r'[\d\s]+', '', x)).unique())

Различные валюты: ['₽']
Различные единицы измерения расстояния: ['км']
Различные единицы измерения двигателя: ['.л/л.с./Бензин' '.л/л.с./Дизель' '.л/л.с./Газ' '.л/л.с./Гибрид'
 'л.с./кВт/Электро']


Как мы видим, с price и mileage все хорошо, а вот с engine придется поработать.

Теперь обработаем параметр engine. Как мы видим, в нем одновременно указан объем, мощьность и вид топлива. Разобъем его на три колонки.

In [10]:
df['engine_capacity'] = np.nan
df['engine_power'] = np.nan
df['fuel_type'] = np.nan

In [11]:
def engine_transform(row):
  try:
    if pd.isna(row['engine']):
      return row
    data = row['engine'].split('/')
    if data[2].strip() == 'Электро':
      data[1] = data[0]
      data[0] = np.nan

    row['engine_capacity'] = data[0]
    row['engine_power'] = data[1]
    row['fuel_type'] = data[2].strip()
    return row
  except:
    print("Ошибка со строкой", row)
    return row

In [12]:
df = df.apply(engine_transform, axis=1)

In [13]:
print(df['engine_capacity'].isna().sum())
print(df['engine_power'].isna().sum())
print(df['fuel_type'].isna().sum())

3606
0
0


Как мы видим, пропусков в новых столбцах нет.

Теперь удалим ненужный столбец engine:

In [14]:
df = df.drop('engine', axis=1)

In [15]:
df.head()

,price,title,year,mileage,transmission,body_type,drive_type,color,engine_capacity,engine_power,fuel_type
3,189 700 ₽,Volkswagen Passat B3,1990,325000км,механика,универсал 5 дв.,передний,красный,1.8 л,90 л.с.,Бензин
4,370 000 ₽,Nissan Terrano I,1990,224000км,автомат,внедорожник 5 дв.,полный,зелёный,2.7 л,100 л.с.,Дизель
5,350 000 ₽,Honda Civic IV,1990,200000км,механика,седан,передний,серебристый,1.6 л,120 л.с.,Бензин
6,360 000 ₽,Volkswagen Passat B3,1990,200000км,механика,универсал 5 дв.,передний,белый,1.8 л,90 л.с.,Бензин
7,99 000 ₽,Toyota Carina V (T170),1990,296000км,автомат,седан,передний,серебристый,1.5 л,94 л.с.,Бензин


В столбец transmission при скрапинге попали данные о дальности хода для электромобилей. Давайте извлечем их оттуда.

In [16]:
import re

df['travel_distance'] = 0


def transmission_transform(row):
  try:
    if pd.isna(row['transmission']):
      return row
    if row['fuel_type'] == 'Электро':
      number = re.findall(r'\d+', row['transmission'])
      if number:
        row['travel_distance'] = int(number[0])
        row['transmission'] = 'none'
        return row
    return row
  except:
    print(row['transmission'])
    return row


df = df.apply(transmission_transform, axis=1)

Посмотрим, какие единицы измерения есть в числовых столбцах, чтобы правильно преобразовать данные:

In [17]:
print("Различные валюты:", df['price'].apply(lambda x: np.nan if pd.isna(x) else re.sub(r'[\d\s]+', '', x)).unique())
print("Различные единицы измерения расстояния:", df['mileage'].apply(lambda x: np.nan if pd.isna(x) else re.sub(r'[\d\s]+', '', x)).unique())
print("Различные единицы измерения объема двигателя:", df['engine_capacity'].apply(lambda x: np.nan if pd.isna(x) else re.sub(r'[\d\s]+', '', x)).unique())
print("Различные единицы измерения мощьности:", df['engine_power'].apply(lambda x: np.nan if pd.isna(x) else re.sub(r'[\d\s]+', '', x)).unique())

Различные валюты: ['₽']
Различные единицы измерения расстояния: ['км']
Различные единицы измерения объема двигателя: ['.л' nan]
Различные единицы измерения мощьности: ['л.с.']


Преобразуем цену в числовую переменную.

In [18]:
def price_to_int(x):
  if pd.isna(x):
    return np.nan
  return int(x.replace('\xa0', "").replace("₽", ""))


In [19]:
df['price'] = df['price'].apply(price_to_int)

Теперь price -- числовая переменная типа int, единица измерения -- рубли.

Преобразуем mileage в числовой формат.

In [20]:
def mileage_to_int(x):
  if pd.isna(x):
    return np.nan
  return int(x.replace('км', '').replace('\xa0', ""))

In [21]:
df['mileage'] = df['mileage'].apply(mileage_to_int)

Теперь mileage -- числовая переменная, единица измерения -- километры.

In [22]:
def engine_capacity_to_int(x):
  if pd.isna(x):
    return np.nan
  return float(x.replace('л', '').strip())

In [23]:
df['engine_capacity'] = df['engine_capacity'].apply(engine_capacity_to_int)

Теперь engine_capacity -- числовая переменная, единицы измерения -- литры.

Преобразуем engine_power в числовой формат.

In [24]:
def engine_power_to_int(x):
  if pd.isna(x):
    return np.nan
  return float(x.replace('л.с.', '').strip())

In [25]:
df['engine_power'] = df['engine_power'].apply(engine_power_to_int)

Теперь engine_power -- числовая переменная, единицы измерения -- лошадиные силы.

In [26]:
df.dtypes

price                int64
title               object
year                 int64
mileage              int64
transmission        object
body_type           object
drive_type          object
color               object
engine_capacity    float64
engine_power       float64
fuel_type           object
travel_distance      int64
dtype: object

Посмотрим на основные статистики наших данных.

In [27]:
df.describe(include='number')

,price,year,mileage,engine_capacity,engine_power,travel_distance
count,1.991820e+05,199182.000000,199182.000000,195576.000000,199182.000000,199182.000000
mean,2.305377e+06,2010.537955,175138.229825,2.315878,188.135630,9.560417
std,5.746065e+06,8.737646,118841.182301,0.899047,110.511373,72.873968
min,2.500000e+04,1990.000000,1.000000,0.400000,10.000000,0.000000
25%,5.600000e+05,2004.000000,81000.000000,1.600000,116.000000,0.000000
50%,1.200000e+06,2011.000000,170000.000000,2.000000,155.000000,0.000000
75%,2.500000e+06,2018.000000,250000.000000,2.700000,231.000000,0.000000
max,1.000000e+09,2025.000000,1000000.000000,8.300000,1548.000000,1055.000000


Как мы видим, отрицательных значений в данных нет. Экстремально больших значений в данных тоже нет.

In [28]:
df.describe(include='object')

,title,transmission,body_type,drive_type,color,fuel_type
count,199182,199182,199182,199182,199176,199182
unique,8182,5,28,3,16,5
top,Lada (ВАЗ) 2110,автомат,внедорожник 5 дв.,полный,чёрный,Бензин
freq,1130,112515,83983,91939,48846,157290


Давайте посмотрим на уникальные значения в данных:

Посмотрим, есть ли в данных пропуски.

In [29]:
df.isna().sum()

price                 0
title                 0
year                  0
mileage               0
transmission          0
body_type             0
drive_type            0
color                 6
engine_capacity    3606
engine_power          0
fuel_type             0
travel_distance       0
dtype: int64

Посмотрим на процент пропусков в данных:

In [30]:
df.isna().mean()*100

price              0.000000
title              0.000000
year               0.000000
mileage            0.000000
transmission       0.000000
body_type          0.000000
drive_type         0.000000
color              0.003012
engine_capacity    1.810405
engine_power       0.000000
fuel_type          0.000000
travel_distance    0.000000
dtype: float64

Пропущенные значения в engine_capacity -- это данные, в которых тип двигателя -- Электро. У такого двигателя нет объема.

Пропусков в цвете всего 5, скорее всего их не указал продавец или при скрапинге страница не успела прогрузить цвет.

In [31]:
df[df['fuel_type'] == 'Электро'].shape

(3606, 12)

color мы заполним самым популярным значением.


In [32]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='most_frequent')
df['color'] = imputer.fit_transform(df[['color']]).ravel()

In [33]:
df['color'].isna().sum()

0

Теперь заполним все пропуски в engine_capacity нулями

In [34]:
df['engine_capacity'] = df['engine_capacity'].fillna(0)

In [35]:
df['engine_capacity'].isna().sum()

0

In [36]:
df.isna().sum().sum()

0

Переносим данные в csv для исследования

In [ ]:
df.to_csv('../full-data/preprocessed.csv')